## **Toy model**: Gradual uniform loading of a 2D unconfined aquifer

In this problem, a spatially uniform compressive stress $-\sigma_o$ is gradually applied from $t = 0$ to $t =  n_t\Delta t/2$ as $\sigma_{yy} = -\sigma_o/(n_t\Delta t/2)t$ over a given fraction of the domain and maintained to $-\sigma_o$ from $t =n_t\Delta t /2$ to $t = n_t\Delta t $. The boundary conditions are similar to Terzaghi's problem except that vertical displacements are set to 0 on the right boundary:

Top boundary:
$$\sigma_{yy}(x,y=H,t) = -\sigma_o/(\Delta tn_t/2)t \text{ if } t<n_t \Delta t/2 \text{  and  } x<r_{load}$$
$$\sigma_{yy}(x,y=H,t) = -\sigma_o \text{  if  } t>=n_t \Delta t/2 \text{ and } x<r_{load}$$
$$\sigma_{yy}(x,y=H,t) = 0 \text{  if  } x>r_{load}$$

$$\sigma_{xy}(x,y=H,t) = 0$$
$$p(x,y=H,t) = 0$$

Bottom boundary: 
$$u(x,y=0,t) = 0$$
$$v(x,y=0,t) = 0$$
$$\frac{\partial p}{\partial y}(x,y=0,t) = 0$$

Left boundary: 
$$u(x=0,y,t) = 0$$
$$\frac{\partial v}{\partial x}(x=0,y,t) = 0$$
$$\frac{\partial p}{\partial x}(x=0,y,t) = 0$$

Right boundary: 
$$u(x=L,y,t) = 0$$
$$v(x=L,y,t) = 0$$
$$\frac{\partial p}{\partial x}(x=L,y,t) = 0$$


In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'Helvetica'
rcParams['font.size'] = 12
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mcolors

import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import inv
from scipy import special as sp
import time

np.set_printoptions(threshold=np.inf)
%matplotlib widget

In [ ]:
## Poroelastic properties:
G = 10e9 # Shear modulus [GPa]
nu = 0.25 # Drained Poisson's ratio
nu_u = 0.35 # Undrained Poisson's ratio
alpha = 0.9 # Biot coefficient
rho_o = 1000 # Reference water density [kg/m^3]
g = 10 # gravitational acceleration
k = 1e-13 # Aquifer permeability [m^2]
mu = 1e-3 # Dynamic viscosity of water [kg/(m·s)] 
c_m = alpha*(1-2*nu)/(2*(1-nu)*G) # Uniaxial poroelastic expansion coefficient
gamma = 1/alpha*(nu_u-nu)/((1-2*nu)*(1-nu_u)) #Loading efficiency
c = k*gamma/(mu*c_m) #hydraulic diffusivity [m^2/s]
B = 3*(nu_u-nu)/(alpha*(1-2*nu)*(1+nu_u)) # Skempton's coefficient
S = k/(mu*c) # Uniaxial specific storage
S_eps = (1-nu)*(1-2*nu_u)/((1-nu_u)*(1-2*nu))*S #Specific storage at constant strain

## Model domain
L = 1000 # Width of domain [m]
H = 1000 # Height of domain [m]

## Numerical parameters:
dt = 1000 # timestep [s]
nt = 150 # Number of timesteps to compute
times = np.arange(dt,(nt+1)*dt,dt) # Time array
n_plot = 10 #Plot every n_plot time step
dx = 25 # Discretization in x-direction
dy = 25 # Discretization in y-direction
x = np.arange(-L/2,L/2+dx,dx) # x gridpoints
y = np.arange(-H,dy,dy) # y gridpoints
[x2d,y2d] = np.meshgrid(x,y) # x and y mesh
nx = len(x) # Number of gridpoints in x direction
ny = len(y) # Number of gridpoints in y direction

## Surface loading:
sigma_o = 1e6 # Amplitude of applied stress [Pa]
sigma_load = -sigma_o/times[nt//2-1]*times #Stress at the surface (negative = compressive loading)
sigma_load[nt//2:nt] = -sigma_o

sigma_surf = np.zeros([nx,nt])
sigma_surf[:nx//4,:] = sigma_load

## Setup arrays for variables to be solved for:
p = np.zeros([nx*ny,1]) # Initial pore pressure [Pa]
u = np.zeros([nx*ny,1]) # Initial horizontal displacements [m]
v = np.zeros([nx*ny,1]) # Initial vertical displacements [m]

## Setup array to verify vertical stress:
sigma_yy = np.zeros((ny-2,1))

## Array of indices:
Number = np.arange(0,nx*ny) 
Number = Number.reshape((nx,ny))

In [ ]:
# Construct A matrix to be inverted:

A1_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of fluid equation
A1_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of fluid equation
A1_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of fluid equation

A2_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of first mechanical equation
A2_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of first mechanical equation
A2_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of first mechanical equation

A3_p = np.zeros((nx*ny,nx*ny)) #Pressure portion of second mechanical equation
A3_u = np.zeros((nx*ny,nx*ny)) #x-displacement portion of second mechanical equation
A3_v = np.zeros((nx*ny,nx*ny)) #y-displacement portion of second mechanical equation


for i in np.arange(0,nx):
    for j in np.arange(0,ny):   

        # Top boundary:
        if j == ny-1: 
            A1_p[Number[i,j],Number[i,j]] = 1 # p = 0 everywhere at top boundary

            # Upper left corner:
            if i == 0:
                #A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G) #p_{i+1,j} - p_{i-1,j} = 0
                #A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                #A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                #A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                #A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i-2,j} = -u_{i,j}
                #A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                #A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy) #v_{i+1,j} - v_{i-1,j} = 0
                
               #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                A3_u[Number[i,j],Number[i+1,j]] = -4*nu/((1-2*nu)*dx*dy) #u_{i+1,j} = -u_{i-1,j}
                #A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy) 
                A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                A3_v[Number[i,j],Number[i+1,j]] = 2/(dx**2) #v_{i+1,j} - v_{i-1,j} = 0
                #A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                #A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2)  #v_{i,j} - v_{i-2,j} = 0  
            
            # Gridpoint to the right of upper left corner (necessary because of the i-2 term):
            elif i == 1:
                A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                #A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i-2,j} = -u_{i,j}
                A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)  
                
               #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                #A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2)  #v_{i,j} - v_{i-2,j} = 0                         
            
            # Gridpoint to the left of upper right corner (necessary because of the i+2 term):
            elif i == nx-2:
                A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                #A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i+2,j} = -u_{i,j}
                A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy) 
                A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)
                
               #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                #A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2) #v_{i+2,j} - v_{i,j} = 0
                     
            # Upper right corner:        
            elif i == nx-1:    
                #A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G) #p_{i+1,j} - p_{i-1,j} = 0
                #A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)           
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))+nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                #A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                #A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2) #u_{i+1,j} = -u_{i-1,j}
                #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                #A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)) # u_{i+2,j} = -u_{i,j}
                A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                #A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy) #v_{i+1,j} - v_{i-1,j} = 0
                #A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)
                 
               #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)  #p_{i,j+1} = - p_{i,j-1}
                A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                #A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                A3_u[Number[i,j],Number[i-1,j]] = 4*nu/((1-2*nu)*dx*dy) #u_{i+1,j} = -u_{i-1,j}
                A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                #A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i-1,j]] = 2/(dx**2) #v_{i+1,j} - v_{i-1,j} = 0
                #A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)-1/((1-2*nu)*4*dx**2) #v_{i+2,j} - v_{i,j} = 0
            
            # Rest of top boundary:
            else:
                A2_p[Number[i,j],Number[i+1,j]] = (2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_p[Number[i,j],Number[i-1,j]] = -(2*nu-1)/(2-2*nu)*alpha/(2*dx*G)
                A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2-nu/((1-2*nu)*(1-nu))*(1/(4*dx**2)))
                A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
                #A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
                A2_u[Number[i,j],Number[i,j-1]] = 2/dy**2
                A2_u[Number[i,j],Number[i+2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                A2_u[Number[i,j],Number[i-2,j]] = -nu/((1-2*nu)*(1-nu))*(1/(4*dx**2))
                A2_v[Number[i,j],Number[i+1,j]] = -1/(dx*dy)
                A2_v[Number[i,j],Number[i-1,j]] = 1/(dx*dy)
                
               #A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G) #p_{i,j+1} = - p_{i,j-1}
                A3_p[Number[i,j],Number[i,j-1]] = 2*alpha/(2*dy*G)
                A3_p[Number[i,j],Number[i,j]] = 2*alpha/(dy*G)
                A3_u[Number[i,j],Number[i+1,j]] = -2*nu/((1-2*nu)*dx*dy)
                A3_u[Number[i,j],Number[i-1,j]] = 2*nu/((1-2*nu)*dx*dy)
                A3_v[Number[i,j],Number[i,j-1]] = 2*(2-2*nu)/((1-2*nu)*dy**2)
                A3_v[Number[i,j],Number[i+1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i-1,j]] = 1/(dx**2)
                A3_v[Number[i,j],Number[i+2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i-2,j]] = -1/((1-2*nu)*4*dx**2)
                A3_v[Number[i,j],Number[i,j]] = -2*(2-2*nu)/((1-2*nu)*dy**2)-2/(dx**2)+1/(2*(1-2*nu)*dx**2)
            
        # Bottom boundary:
        elif j == 0:
            
            A2_u[Number[i,j],Number[i,j]] = 1
            A3_v[Number[i,j],Number[i,j]] = 1
            
            # Bottom left corner: 
            if i == 0:
                A1_p[Number[i,j],Number[i+1,j]] = -2*(k/(alpha*mu))/dx**2  #p_{i+1,j} - p_{i-1,j} = 0
                #A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                A1_u[Number[i,j],Number[i+1,j]] = 2/(2*dt*dx) #u_{i+1,j} = -u_{i-1,j}
                #A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   
             
            # Bottom right corner:
            elif i == nx-1:
                #A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i-1,j]] = -2*(k/(alpha*mu))/dx**2  #p_{i+1,j} - p_{i-1,j} = 0
                A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                #A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx) #u_{i+1,j} = -u_{i-1,j}
                A1_u[Number[i,j],Number[i-1,j]] = -2/(2*dt*dx)
                A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   
                
            # Rest of bottom boundary:    
            else: 
                A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
                A1_p[Number[i,j],Number[i,j+1]] = -2*(k/(alpha*mu))/dy**2 #p_{i,j+1} - p_{i,j-1} = 0
                #A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2 
                A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
                A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx)
                A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
                A1_v[Number[i,j],Number[i,j+1]] = 2/(2*dt*dy) #v_{i,j+1} = -v_{i-1,j}
                #A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)      
        
        # Left boundary:
        elif i == 0: 
            A1_p[Number[i,j],Number[i+1,j]] = -2*(k/(alpha*mu))/dx**2 #p_{i+1,j} - p_{i-1,j} = 0
            #A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
            A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
            A1_u[Number[i,j],Number[i+1,j]] = 2/(2*dt*dx)  #u_{i+1,j} = -u_{i-1,j}
            #A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
            A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
            A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)   
            
            A2_u[Number[i,j],Number[i,j]] = 1
            
            A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)
            A3_p[Number[i,j],Number[i,j-1]] = alpha/(2*dy*G)
            A3_v[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dy**2)+1/dx**2)
            A3_v[Number[i,j],Number[i+1,j]] = 2/dx**2 #v_{i+1,j} - v_{i-1,j} = 0
            #A3_v[Number[i,j],Number[i-1,j]] = 1/dx**2
            A3_v[Number[i,j],Number[i,j+1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
            A3_v[Number[i,j],Number[i,j-1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
            A3_u[Number[i,j],Number[i+1,j+1]] = 2/(1-2*nu)*(1/(4*dx*dy)) #u_{i+1,j} = -u_{i-1,j}
            A3_u[Number[i,j],Number[i+1,j-1]] = -2/(1-2*nu)*(1/(4*dx*dy)) #u_{i+1,j} = -u_{i-1,j}
            #A3_u[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
            #A3_u[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))   
        
        # Right boundary:
        elif i == nx-1:
            #A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2 
            A1_p[Number[i,j],Number[i-1,j]] = -2*(k/(alpha*mu))/dx**2 #p_{i+1,j} - p_{i-1,j} = 0
            A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
            #A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx) 
            A1_u[Number[i,j],Number[i-1,j]] = -2/(2*dt*dx)  #u_{i+1,j} = -u_{i-1,j}
            A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
            A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)      
            
            A2_u[Number[i,j],Number[i,j]] = 1
            A3_v[Number[i,j],Number[i,j]] = 1

        # Domain interior: 
        else: 
            A1_p[Number[i,j],Number[i+1,j]] = -(k/(alpha*mu))/dx**2
            A1_p[Number[i,j],Number[i-1,j]] = -(k/(alpha*mu))/dx**2
            A1_p[Number[i,j],Number[i,j+1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j-1]] = -(k/(alpha*mu))/dy**2
            A1_p[Number[i,j],Number[i,j]] = (2*k/(alpha*mu))*(1/dx**2+1/dy**2)+S_eps/(alpha*dt)
            A1_u[Number[i,j],Number[i+1,j]] = 1/(2*dt*dx)
            A1_u[Number[i,j],Number[i-1,j]] = -1/(2*dt*dx)
            A1_v[Number[i,j],Number[i,j+1]] = 1/(2*dt*dy)
            A1_v[Number[i,j],Number[i,j-1]] = -1/(2*dt*dy)     
            
            A2_p[Number[i,j],Number[i+1,j]] = -alpha/(2*dx*G)
            A2_p[Number[i,j],Number[i-1,j]] = alpha/(2*dx*G)
            A2_u[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dx**2)+1/dy**2)
            A2_u[Number[i,j],Number[i+1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
            A2_u[Number[i,j],Number[i-1,j]] = (2-2*nu)/(1-2*nu)*(1/dx**2)
            A2_u[Number[i,j],Number[i,j+1]] = 1/dy**2
            A2_u[Number[i,j],Number[i,j-1]] = 1/dy**2
            A2_v[Number[i,j],Number[i+1,j+1]] = 1/(1-2*nu)*(1/(4*dx*dy))
            A2_v[Number[i,j],Number[i+1,j-1]] = -1/(1-2*nu)*(1/(4*dx*dy))
            A2_v[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
            A2_v[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))

            A3_p[Number[i,j],Number[i,j+1]] = -alpha/(2*dy*G)
            A3_p[Number[i,j],Number[i,j-1]] = alpha/(2*dy*G)
            A3_v[Number[i,j],Number[i,j]] = -2*((2-2*nu)/(1-2*nu)*(1/dy**2)+1/dx**2)
            A3_v[Number[i,j],Number[i+1,j]] = 1/dx**2
            A3_v[Number[i,j],Number[i-1,j]] = 1/dx**2
            A3_v[Number[i,j],Number[i,j+1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
            A3_v[Number[i,j],Number[i,j-1]] = (2-2*nu)/(1-2*nu)*(1/dy**2)
            A3_u[Number[i,j],Number[i+1,j+1]] = 1/(1-2*nu)*(1/(4*dx*dy))
            A3_u[Number[i,j],Number[i+1,j-1]] = -1/(1-2*nu)*(1/(4*dx*dy))
            A3_u[Number[i,j],Number[i-1,j+1]] = -1/(1-2*nu)*(1/(4*dx*dy))
            A3_u[Number[i,j],Number[i-1,j-1]] = 1/(1-2*nu)*(1/(4*dx*dy))   

# Concatenate the submatrices into one big matrix to be inverted:            
A1 = np.concatenate((A1_p, A1_u, A1_v), axis=1)
A2 = np.concatenate((A2_p, A2_u, A2_v), axis=1)
A3 = np.concatenate((A3_p, A3_u, A3_v), axis=1)
A = np.concatenate((A1, A2, A3),axis=0)

# Compute inverse of time-independent matrix A: 
start_time = time.time()

A_sparse = csr_matrix(A)
A_inv = inv(A_sparse) 

time_inv = time.time() - start_time

print('Matrix A took ' + str(time_inv) + ' s to invert')

In [ ]:
### Time loop to compute solution m given the linear system of equations A*m = b at each time step:

## Initialize figures:
fig2, ax2 = plt.subplots(1,3,figsize = (12,4),constrained_layout=True)

## Main time loop, plotting results every n_plot timestep: 
for n in np.arange(0,nt):    
      
    # Initial b-vectors containing the RHS of each equation:
    
    b1 = np.zeros([nx*ny,1]) # b vector for fluid equation
    b2 = np.zeros([nx*ny,1]) # b vector for first mechanical equation
    b3 = np.zeros([nx*ny,1]) # b vector for second mechanical equation

    for i in np.arange(0,nx):
        for j in np.arange(0,ny):   

            # Top boundary:
            if j == ny-1: 
                b1[Number[i,j]] = 0 
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = -2*sigma_surf[i,n]/(G*dy)
            
            # Bottom boundary:
            elif j == 0:
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
                
                # Bottom left corner:
                if i == 0:
                     b1[Number[i,j]] = 2/(2*dx*dt)*(u[Number[i+1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  
                
                # Bottom right corner:
                elif i == nx-1:
                     b1[Number[i,j]] = 2/(2*dx*dt)*(-u[Number[i-1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  

                # Rest of bottom boundary:        
                else: 
                     b1[Number[i,j]] = 1/(2*dx*dt)*(u[Number[i+1,j]]-u[Number[i-1,j]])+2/(2*dy*dt)*(v[Number[i,j+1]])+S_eps/(alpha*dt)*p[Number[i,j]]  

            # Left boundary:            
            elif i == 0: 
                b1[Number[i,j]] = 1/(2*dx*dt)*(2*u[Number[i+1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
           
            # Right boundary
            elif i == nx-1:              
                b1[Number[i,j]] = 1/(2*dx*dt)*(-2*u[Number[i-1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]              
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0
                
            # Domain interior: 
            else: 
                b1[Number[i,j]] = 1/(2*dx*dt)*(u[Number[i+1,j]]-u[Number[i-1,j]])+1/(2*dy*dt)*(v[Number[i,j+1]]-v[Number[i,j-1]])+S_eps/(alpha*dt)*p[Number[i,j]]
                b2[Number[i,j]] = 0
                b3[Number[i,j]] = 0    

    b = np.concatenate((b1, b2, b3),axis=0)
    
    # Solve for solution vector m:
    m_sol = A_inv*b
    
    # Update p, u, and v: 
    p = m_sol[Number.flatten()]
    u = m_sol[Number.flatten()+nx*ny]
    v = m_sol[Number.flatten()+2*nx*ny]
    
    p_array = p.reshape((nx,ny))
    v_array = v.reshape((nx,ny))
    u_array = u.reshape((nx,ny))
   
    if n % n_plot == 0:
        fig, ax = plt.subplots(1,3,figsize = (15,4),constrained_layout=True)
        p1 = ax[0].pcolor(x2d/1e3,y2d/1e3,p_array.T/1e6,cmap='YlGnBu',shading = 'auto',vmin=0,vmax=0.10)
        ax[0].contour(x2d/1e3, y2d/1e3, p_array.T/1e6, levels=np.arange(0,0.35,0.01),colors='k',linewidths=0.5)
        ax[0].set_xlabel("x [km]")
        ax[0].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p1,ax=ax[0],aspect = 20)
        cb.set_label(label='$p$ [MPa]')
        #cb.ax.tick_params(labelsize=16)
        
        p2 = ax[1].pcolor(x2d/1e3,y2d/1e3,v_array.T*1e3,cmap='RdBu_r',shading = 'auto',vmin=-30,vmax=30)
        ax[1].contour(x2d/1e3, y2d/1e3, v_array.T*1e3, levels=np.arange(-30,30,1),colors='k',linewidths=0.5)
        ax[1].set_xlabel("x [km]")
        ax[1].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p2,ax=ax[1],aspect = 20)
        cb.set_label(label='$v$ [mm]')
        #cb.ax.tick_params(labelsize=16)

        p3 = ax[2].pcolor(x2d/1e3,y2d/1e3,u_array.T*1e3,cmap='RdBu_r',shading = 'auto',vmin=-5,vmax=5)
        #ax[2].contour(x2d/1e3, y2d/1e3, u_array.T*1e3, levels=10,colors='k',linewidths=0.5)
        ax[2].set_xlabel("x [km]")
        ax[2].set_ylabel("y [km]")
        #ax[0].set_title('Time = '+str(round(time/3600/24))+' days')
        cb = plt.colorbar(p3,ax=ax[2],aspect = 20)
        cb.set_label(label='$u$ [mm]')
        #cb.ax.tick_params(labelsize=16)
        
        ax2[0].plot(p_array[0,:]/1e6,y,'.')
        ax2[0].set_xlabel("p [MPa]")
        ax2[0].set_ylabel("y [km]")
        ax2[0].set_title("Pore pressure")
        
        ax2[1].plot(x/1e3,v_array[:,ny-1]*1e3,'.')
        ax2[1].set_xlabel("x [km]")
        ax2[1].set_ylabel("v [mm]")
        ax2[1].set_title("Vertical displacement")
        
        ax2[2].plot(x/1e3,u_array[:,ny-1]*1e3,'.')
        ax2[2].set_xlabel("x [km]")
        ax2[2].set_ylabel("u [mm]")
        ax2[2].set_title("Horizontal displacement")